In [1]:
import pandas
import numpy as np
import import_ipynb
import FeatureStore

importing Jupyter notebook from FeatureStore.ipynb


In [2]:
from sklearn.preprocessing import StandardScaler

#https://stackoverflow.com/questions/35723472/how-to-use-sklearn-fit-transform-with-pandas-and-return-dataframe-instead-of-num
def scaleDataFrame(dataframe, debugPrint = False):
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(dataframe)
    dataframe = pandas.DataFrame(scaled_features, index=dataframe.index, columns=dataframe.columns)
    
    #check
    if debugPrint == True:
        for column in dataframe:
            print("[%s] mean = %.4f, variance = %.4f" % 
                  (column, dataframe[column].mean(), dataframe[column].var()))
    
    return dataframe, scaler.mean_, scaler.scale_

In [3]:
results = {}
for period in [1, 2, 3, 5, 10, 20, 40, 60, 120, 250]:
    results[period] = {}
    for trees in [100, 200, 300]:
        results[period][trees] = {}
        for criterion in ['gini', 'entropy']:
            results[period][trees][criterion] = 0.0

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

from sklearn.model_selection import cross_val_score

from datetime import datetime

Alpha = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
#C = {1.0}
#grid = {'C': np.power(10.0, np.arange(-4, 0))}

sigma1 = 0
mu1 = 0
clf1 = 0
accs = {}


            
for period in [1, 2, 3, 5, 10, 20, 40, 60, 120, 250]:
    X_train = FeatureStore.getTrainDataset(period)
    y_train = FeatureStore.getTrainTarget(period)
    
    #print(X_train.shape, y_train.shape)
    X_train, sigma, mu = scaleDataFrame(X_train, debugPrint = False)
    #sigma = 1.0
    #mu = 0.0
    #print(sigma)
    #print(mu)

    for trees in [100, 200, 300]:
        for criterion in ['gini', 'entropy']:
            print("trying %d-tree forest, period = %d, function = %s" % (trees, period, criterion))
            kf = KFold(n_splits=3, shuffle=False, random_state=241)

            clf = RandomForestClassifier(n_estimators=trees, criterion = criterion, random_state=241)
            if (period == 40):
                sigma1 = sigma
                mu1 = mu
                clf1 = clf
            #N = 800
            #XX = X_train.iloc[:N, :]
            #yy = y_train[:N]
            XX = X_train
            yy = y_train
            #print("from", datetime.now())
            scores = cross_val_score(clf, XX, yy, scoring='accuracy', cv=kf)
            
            #print("spent iterations %d" % (clf.n_iter_))
            #print(clf)
            #print(vars(clf))
            #print(clf.n_iter_)
            #print(scores)
            print("accuracy=", scores.mean())
            results[period][trees][criterion] = scores.mean() 
            #print("to", datetime.now())
            #accs[period].append(scores.mean())
            if period in accs and not (accs[period] is None):
                accs[period].append(scores.mean())
            else:
                accs[period] = [scores.mean()]
        
for k in accs:
    accs[k] = np.max(accs[k])

#print(accs)

trying 100-tree forest, period = 1, function = gini
accuracy= 0.7596977208344065
trying 100-tree forest, period = 1, function = entropy
accuracy= 0.7615943428620482
trying 200-tree forest, period = 1, function = gini
accuracy= 0.7540239505536955
trying 200-tree forest, period = 1, function = entropy
accuracy= 0.7644298866855523
trying 300-tree forest, period = 1, function = gini
accuracy= 0.7540185852862907
trying 300-tree forest, period = 1, function = entropy
accuracy= 0.7663184608120869
trying 100-tree forest, period = 2, function = gini
accuracy= 0.9990557129367327
trying 100-tree forest, period = 2, function = entropy
accuracy= 0.9990557129367327
trying 200-tree forest, period = 2, function = gini
accuracy= 0.9990557129367327
trying 200-tree forest, period = 2, function = entropy
accuracy= 0.9990557129367327
trying 300-tree forest, period = 2, function = gini
accuracy= 0.9990557129367327
trying 300-tree forest, period = 2, function = entropy
accuracy= 0.9990557129367327
trying 100

In [5]:
for k, v in accs.items():
    print("%d;%f" % (k, v))
    

print(clf1, sigma1, mu1)
print(results)

import json
with open('resultRF.json', 'w') as fp:
    json.dump(results, fp)

1;0.766318
2;0.999056
3;0.886482
5;0.802279
10;0.745496
20;0.751180
40;0.789032
60;0.790918
120;0.873211
250;0.934726
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=241,
                       verbose=0, warm_start=False) [1.05107978e-03 9.80585321e-04 1.37135076e+08 1.37707874e+08
 2.03311258e+00 1.56745506e+01] [1.27345881e-02 1.29274745e-02 6.57771957e+07 6.61035530e+07
 1.40476317e+00 8.69552487e+00]
{1: {100: {'gini': 0.7596977208344065, 'entropy': 0.7615943428620482}, 200: {'gini': 0.7540239505536955, 'entropy': 0.7644298866855523}, 300: {'gini': 0.7540185852862907, 'entropy': 0.7663184608120869}},

In [6]:
def backtestStrategy():
    pass